## Измерение магнитного поля земли

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp

#### Период колебаний стержня 

In [2]:
vibQ:int = 5 #количество колебаний в каждом эксперименте
time = np.array([36,36,40]) #Время для 5 колебаний зайчика
timeError = 0.5 #в качестве ошибки измерения времени(человеческий фактор+ прибор) возьмем половину секунды
period: float = np.mean(time/vibQ) #период колебаний - среднее значение по всем экспериментам
periodMSE: float = np.sqrt(np.sum(np.power((time/vibQ)-period,2))/len(time)) #срежне квадратичное отклонение
periodError2: float = timeError/vibQ #ошибка, связанная с человеским фактором+погрешность прибора
periodError: float = np.sqrt(periodMSE**2+periodError2**2) #сложение инструментальной погрешности и средне квадратичной ошибки по экспериментам
print("Период колбеаний зайчика: " + str(period)+" +/- "+str( periodError))

Период колбеаний зайчика: 7.466666666666666 +/- 0.3901566636906541


#### Прочие данные эксперимента

In [74]:
L_n,LError_n = 0.86,0.01 #расстояние от шкалы до зеркала, погрешность - один сантиметр(погрешность прибора(линейки)+учитываем ширину самой шкалы(около 5 мм))
l_n,lError_n = 0.04,0.0005 #линейный размер намагниченного стержня + ошибка(использовался штангенциркуль)
m_n,mError_n = 5.9*np.power(0.1,3),0.1*np.power(0.1,3) #масса стержня + ошибка(электронные весы)
R_n,RError_n = 0.25,0.005 #радиус окружности(с витками внутри)
x1_n,x1Error_n = 0.04,0.005 #отклонение зайчика в поле намгниченногос тержня
r_n,rError_n = 0.005,0.0005 #радиус стержня + ошибка(использовался штангенциркуль)
mu_n = 4 * np.pi * np.power(0.1,7) #магнитная постоянная

#### Рассчет горизонатльной состовляющей магнитного поля земли

In [76]:
T,R,mu,J,L,x1 = smp.symbols("T R \mu_{0} J L x_{1}") #вводим символы, использующиеся в работе
dT,dR,dJ,dL,dx1 = smp.symbols(r"\Delta_{T} \Delta_{R} \Delta_{J} \Delta_{L} \Delta_{x_{1}}") #символы ошибок
m,l,r  = smp.symbols(r"m l r")
dm,dl,dr = smp.symbols(r"\Delta_{m} \Delta_{l} \Delta_{r}")
#Рассчитаем момент инерции
J = m*(((l**2)/12)+((r**2)/(4)))
B0 = 2*smp.pi/(T*R)
B0*=smp.sqrt(mu*J*L/(2*smp.pi*R*x1))
B0variables = [m,l,r,R,T,L,x1] #параметры, от которыйх зависит B0
B0ErrorVariablesA = [dm,dl,dr,dR,dT,dL,dx1] 
B0ErrorVariables = smp.Matrix(B0ErrorVariablesA) #столбец, содержащий ошибки по всем переменным от которых зависит наша B0
B0gradient = smp.Matrix([B0.diff(i) for i in B0variables])
B0ErrorsVector = smp.Matrix([B0gradient[i]*B0ErrorVariables[i] for i in range(len(B0ErrorVariables))]) #вектор, содержащий ошибки вычсиления B0 по кажой из переменных
dB0 = B0ErrorsVector.norm() #норма вектора - и есть искомая ошибка
dB0 #выражение для вычисления ошибки B0

sqrt(9*pi*Abs(\Delta_{R}*sqrt(L*\mu_{0}*m*(l**2/12 + r**2/4)/(R*x_{1}))/(R**2*T))**2/2 + 2*pi*Abs(\Delta_{T}*sqrt(L*\mu_{0}*m*(l**2/12 + r**2/4)/(R*x_{1}))/(R*T**2))**2 + pi*Abs(\Delta_{L}*sqrt(L*\mu_{0}*m*(l**2/12 + r**2/4)/(R*x_{1}))/(L*R*T))**2/2 + pi*Abs(\Delta_{m}*sqrt(L*\mu_{0}*m*(l**2/12 + r**2/4)/(R*x_{1}))/(R*T*m))**2/2 + pi*Abs(\Delta_{x_{1}}*sqrt(L*\mu_{0}*m*(l**2/12 + r**2/4)/(R*x_{1}))/(R*T*x_{1}))**2/2 + pi*Abs(\Delta_{l}*l*sqrt(L*\mu_{0}*m*(l**2/12 + r**2/4)/(R*x_{1}))/(R*T*(l**2/12 + r**2/4)))**2/72 + pi*Abs(\Delta_{r}*r*sqrt(L*\mu_{0}*m*(l**2/12 + r**2/4)/(R*x_{1}))/(R*T*(l**2/12 + r**2/4)))**2/8)

In [75]:
B0_n = smp.lambdify(B0variables+[mu],B0)(m_n,l_n,r_n,R_n,period,L_n,x1_n,mu_n) #вычисление B0
B0Error_n = smp.lambdify(B0variables+B0ErrorVariablesA+[mu],dB0)(m_n,l_n,r_n,R_n,period,L_n,x1_n,mError_n,lError_n,rError_n,RError_n,periodError,LError_n,x1Error_n,mu_n) #вычисление погреншности B0
print("Горизонтальная составляющая магнитного поля земли: "+str(B0_n*np.power(10,9))+" +/- "+str(B0Error_n*np.power(10,9))+" нТл ")

Горизонтальная составляющая магнитного поля земли: 12668.351589637483 +/- 1119.1884063813147 нТл 


### Измерение электродинамической постоянной

#### Данные работы

In [77]:
N_n: int = 34
C_n = 9*np.power(0.1,5)
CErr_n = 0.02*C_n
U_n,UError_n = 95,95*0.01
x_n,xError_n = np.mean(np.array([11.5,12])/100),0.005
nu_n = 50

##### Рассчёт тока в единицах си

In [85]:
I, B0, R, x, L, N,nu = smp.symbols(r"I B_{0} R x L N \nu")
dI,dB0,dR,dx,dL = smp.symbols(r"\Delta_{I} \Delta_{B_{0}} \Delta_{R} \Delta_{x} \Delta_{L}")
I = x*(2*B0*R)/(2*L*mu*N)
IVariables = [B0,R,x,L]
IErrorVariables = [dB0, dR ,dx, dL]
Igradient = smp.Matrix([I.diff(IVariables[i]) for i in range(len(IVariables))])
IErrorVector = smp.Matrix([Igradient[i]*IErrorVariables[i] for i in range(len(IErrorVariables))])
dI = IErrorVector.norm()


In [92]:
I_n = smp.lambdify(IVariables+[N,mu],I)(B0_n,R_n,x_n,L_n,N_n,mu_n)
IError_n = smp.lambdify(IVariables+IErrorVariables+[N,mu],dI)(B0_n,R_n,x_n,L_n,B0Error_n,RError_n,xError_n,LError_n,N_n,mu_n)
print("Сила тока в единицах си: "+str(I_n)+" +/- "+str(IError_n)+" A ")

Сила тока в единицах си: 0.010127698196701127 +/- 0.0010203806260523462 A 


##### Рассчет силы тока в единицах СГС

In [ ]:
C,U,nu,I2 = smp.symbols("C U \nu I_{2}")
I2 = C*U*nu
dC,dU 